In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [7]:
def clean_bkref_dataframe(df):
    """ Cleans dataframes we get from importing bkref CSV files.
    """
    df = df.drop(columns=['Unnamed: 29'])
    df = df.drop(columns=['Rk'])
    df = df.drop_duplicates(subset=['Player'])
    df['Year'] = 2019
    df[['Player', 'player_id']] = df['Player'].str.split('\\', expand=True)
    df = df.reset_index(drop=True)
    return df

In [8]:
players19 = pd.read_csv("data/2019-1-12.csv")

In [9]:
players19 = clean_bkref_dataframe(players19)

In [10]:
players19.to_json("data/2019-per-100.json")

In [11]:
player_df = pd.read_json("data/1980-2018-per100-labeled.json")

In [15]:
team_df = pd.read_csv("data/team-stats-clean.csv")

In [19]:
team_df = team_df[team_df.YEAR >= 1980]

In [24]:
team_df = team_df.drop(columns=['Unnamed: 0'])

In [28]:
team_df.columns = ['Tm', 'Year', 'GP', 'WINS', 'LOSSES', 'WIN_PCT', 'CONF_RANK',
       'DIV_RANK', 'PO_WINS', 'PO_LOSSES', 'CONF_COUNT', 'DIV_COUNT',
       'NBA_FINALS_APPEARANCE', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'PF',
       'STL', 'TOV', 'BLK', 'PTS', 'PTS_RANK']

In [30]:
team_df.to_json("data/team-stats.json")

In [31]:
team_df

,Tm,Year,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,PO_WINS,PO_LOSSES,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
30,ATL,1980,82,50,32,0.610,2,1,1,4,...,1369,2406,3775,1913,2293,782,1495,539,8573,19
31,ATL,1981,82,31,51,0.378,8,4,0,0,...,1201,2224,3425,1846,2276,749,1605,469,8604,19
32,ATL,1982,82,42,40,0.512,6,2,0,2,...,1135,2368,3503,1815,2268,608,1343,485,8281,23
33,ATL,1983,82,43,39,0.524,6,2,1,2,...,1139,2433,3572,1945,2022,573,1424,665,8335,19
34,ATL,1984,82,40,42,0.488,7,3,2,3,...,1112,2232,3344,1827,2091,626,1329,558,8321,23
35,ATL,1985,82,34,48,0.415,9,5,0,0,...,1161,2345,3506,2009,2047,665,1475,541,8743,20
36,ATL,1986,82,50,32,0.610,4,2,4,5,...,1249,2405,3654,2025,2170,736,1483,434,8906,17
37,ATL,1987,82,57,25,0.695,2,1,4,5,...,1350,2478,3828,2077,2152,700,1279,511,9024,12
38,ATL,1988,82,50,32,0.610,3,2,6,6,...,1228,2379,3607,2062,2050,635,1225,537,8844,13
39,ATL,1989,82,52,30,0.634,4,3,2,3,...,1372,2316,3688,1990,1880,817,1310,474,9102,9


In [34]:
player_team_combined = pd.merge(player_df, team_df, on=['Tm','Year'])

In [35]:
player_team_combined.to_json("data/combined-player-team-stats.json")

In [37]:
player_team_combined.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA_x', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA_x', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST_x', 'STL_x', 'BLK_x', 'TOV_x', 'PF_x', 'PTS_x',
       'ORtg', 'DRtg', 'Year', 'MVP', 'player_id', 'GP', 'WINS', 'LOSSES',
       'WIN_PCT', 'CONF_RANK', 'DIV_RANK', 'PO_WINS', 'PO_LOSSES',
       'CONF_COUNT', 'DIV_COUNT', 'NBA_FINALS_APPEARANCE', 'FGM', 'FGA_y',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA_y', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST_y', 'PF_y', 'STL_y', 'TOV_y', 'BLK_y', 'PTS_y',
       'PTS_RANK'],
      dtype='object')

In [38]:
player_df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,STL,BLK,TOV,PF,PTS,ORtg,DRtg,Year,MVP,player_id
0,Quincy Acy,PF,24,NYK,68,22.0,1287,6.2,13.5,0.459,...,1.1,0.9,2.5,6.0,16.3,105,109,2015,0,acyqu01
1,Steven Adams,C,21,OKC,70,67.0,1771,6.1,11.3,0.544,...,1.1,2.4,2.8,6.3,15.2,108,104,2015,0,adamsst01
10,Lou Amundson,PF,32,TOT,53,35.0,937,6.1,14.3,0.427,...,1.2,2.9,3.5,7.1,14.5,92,106,2015,0,amundlo01
100,DeMar DeRozan,SG,25,TOR,60,60.0,2100,10.1,24.4,0.413,...,1.8,0.3,3.4,3.0,29.6,104,109,2015,0,derozde01
1000,Larry Smith,PF,32,HOU,74,0.0,1300,3.7,7.8,0.474,...,2.0,1.0,2.6,7.4,8.1,100,101,1990,0,smithla01
10000,Zoran Planinic,SG,21,NJN,49,1.0,473,6.0,14.6,0.411,...,1.5,0.3,4.1,7.9,17.4,96,101,2004,0,planizo01
10001,Scot Pollard,C,28,IND,61,3.0,678,3.8,9.2,0.412,...,1.9,2.1,1.8,8.9,8.6,96,96,2004,0,pollasc01
10002,James Posey,SG,27,MEM,82,82.0,2451,7.9,16.5,0.478,...,2.9,0.9,2.4,5.3,24.1,119,101,2004,0,poseyja01
10003,Vitaly Potapenko,C,28,SEA,65,39.0,1419,7.5,15.4,0.489,...,0.8,1.1,2.9,6.1,17.3,100,110,2004,0,potapvi01
10004,Tayshaun Prince,SF,23,DET,82,80.0,2701,6.8,14.6,0.467,...,1.3,1.4,2.4,3.0,17.0,106,98,2004,0,princta01


In [40]:
player_df19 = pd.read_json("data/2019-per-100.json")

In [42]:
combined_19 = pd.merge(player_df19, team_df, on=['Tm','Year'])

In [44]:
combined_19.to_json("data/2019-combined.json")

In [ ]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [6]:
urllib.request.urlretrieve("https://www.basketball-reference.com/leagues/NBA_2019_per_poss.html", filename="data/2019_per_poss.html")

('data/2019_per_poss.html', <http.client.HTTPMessage at 0x105d4c208>)

In [8]:
with open("data/2019_per_poss.html") as html_doc:
    soup = BeautifulSoup(html_doc, 'html.parser')

In [14]:
table = soup.find("table")

In [15]:
table_body = table.find('tbody')

rows = table_body.find_all('tr')

In [16]:
rows

[<tr class="full_table"><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Abrines,Alex" data-append-csv="abrinal01" data-stat="player"><a href="/players/a/abrinal01.html">Alex Abrines</a></td><td class="center" data-stat="pos">SG</td><td class="right" data-stat="age">25</td><td class="left" data-stat="team_id"><a href="/teams/OKC/2019.html">OKC</a></td><td class="right" data-stat="g">29</td><td class="right" data-stat="gs">2</td><td class="right" data-stat="mp">573</td><td class="right" data-stat="fg_per_poss">4.5</td><td class="right" data-stat="fga_per_poss">12.5</td><td class="right" data-stat="fg_pct">.359</td><td class="right" data-stat="fg3_per_poss">3.3</td><td class="right" data-stat="fg3a_per_poss">10.1</td><td class="right" data-stat="fg3_pct">.323</td><td class="right" data-stat="fg2_per_poss">1.2</td><td class="right" data-stat="fg2a_per_poss">2.4</td><td class="right" data-stat="fg2_pct">.517</td><td class="right" data-stat="ft_per_poss">

In [18]:
data = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) 

In [19]:
data

[['Alex Abrines',
  'SG',
  '25',
  'OKC',
  '29',
  '2',
  '573',
  '4.5',
  '12.5',
  '.359',
  '3.3',
  '10.1',
  '.323',
  '1.2',
  '2.4',
  '.517',
  '1.0',
  '1.1',
  '.923',
  '0.4',
  '3.4',
  '3.8',
  '1.6',
  '1.4',
  '0.5',
  '1.1',
  '4.2',
  '13.2',
  '104',
  '108'],
 ['Quincy Acy',
  'PF',
  '28',
  'PHO',
  '3',
  '0',
  '26',
  '0.0',
  '3.7',
  '.000',
  '0.0',
  '3.7',
  '.000',
  '0.0',
  '0.0',
  '1.9',
  '3.7',
  '.500',
  '0.0',
  '1.9',
  '1.9',
  '0.0',
  '0.0',
  '0.0',
  '1.9',
  '11.2',
  '1.9',
  '28',
  '121'],
 ['Jaylen Adams',
  'PG',
  '22',
  'ATL',
  '8',
  '0',
  '27',
  '3.4',
  '10.2',
  '.333',
  '3.4',
  '8.5',
  '.400',
  '0.0',
  '1.7',
  '.000',
  '0.0',
  '0.0',
  '0.0',
  '5.1',
  '5.1',
  '6.8',
  '1.7',
  '1.7',
  '1.7',
  '0.0',
  '10.2',
  '108',
  '113'],
 ['Steven Adams',
  'C',
  '25',
  'OKC',
  '41',
  '41',
  '1392',
  '8.8',
  '14.6',
  '.601',
  '0.0',
  '0.0',
  '.000',
  '8.8',
  '14.6',
  '.602',
  '3.6',
  '6.4',
  '.552',
  